# Testing/debugging PQMF

## Warning: outdated

In [ ]:
%load_ext autoreload
%autoreload 2

import torch
import tensorflow as tf

from thesis import pqmf
from ddsp.colab.colab_utils import (
    auto_tune, get_tuning_factor, download,
    play, record, specplot, upload, audio_bytes_to_np,
    DEFAULT_SAMPLE_RATE)
import numpy as np
import IPython

def play_audio(audio):
    audio = np.array(audio)
    audio = np.squeeze(audio)
    IPython.display.display(IPython.display.Audio(audio, rate=16000))

from ddsp.losses import SpectralLoss

def get_loss(target_audio, audio):
    loss_class = SpectralLoss()
    loss = loss_class(tf.convert_to_tensor(target_audio), tf.convert_to_tensor(audio))
    return float(loss)

In [ ]:
sample_rate = 16000
bands = 8
attenuation = 100

input_f = open("../data/audio/violin/II. Double.mp3", "rb")
wav_bytes = input_f.read()
audio = audio_bytes_to_np(wav_bytes)
audio = audio[:sample_rate * 10]
audio_np = audio.copy()

if len(audio.shape) == 1:
    audio = audio[np.newaxis, :, np.newaxis]

audio_tf = tf.convert_to_tensor(audio)

In [ ]:
play_audio(audio[0,:,0])

In [ ]:
p = pqmf.PQMF(pqmf.MultiBandMelGANGeneratorConfig(subbands=8, beta=9, cutoff_ratio=0.05))

In [ ]:
p.synthesis_filter.shape

In [ ]:
%%timeit -n100
# Analysis
a = p.analysis(audio_tf)
# print("Analyzed to", a.shape)

rec_tf = p.synthesis(a)
# print("Synthesized to", rec_tf.shape)

# play_audio(rec_tf[0,:,0])
# get_loss(audio_tf[:,:,0], rec_tf[:,:,0])

In [ ]:
# Upsample to hear what it sounds like

from thesis.newt import resample

a2 = resample(a, a.shape[1] * 8)

a2.shape

for i in range(8):
    IPython.display.display(IPython.display.Audio(0.5 * a2[:,:sample_rate*10,i], rate=sample_rate))

In [ ]:
specplot(audio[0])

In [ ]:
# Show spectrograms to visualize
for i in range(8):
    specplot(a2[:,:,i])

In [ ]:
import torch
from thesis import pqmf0

audio_torch = torch.Tensor(audio_np).reshape((1, 1, -1))

p0 = pqmf0.PQMF(attenuation=attenuation, n_band=bands, polyphase=False)

In [ ]:
# %%timeit
temp0 = p0.forward(audio_torch)
print("Analyzed to", temp0.shape)
rec0 = p0.inverse(temp0)
# print("Synthesized to", rec_torch.shape)

In [ ]:
# Upsample to hear what it sounds like

from thesis.newt import resample

a2 = resample(temp0.swapaxes(1, 2), temp0.shape[2] * 8)

print(a2.shape)

for i in range(8):
    play_audio(a2[:,:sample_rate*3, i])

In [ ]:
temp0 = p0.forward(audio_torch)
print("Analyzed to", temp0.shape)
rec_torch = p0.inverse(temp0)
print("Synthesized to", rec_torch.shape)

play_audio(rec_torch)

get_loss(audio_torch[0], rec_torch[0])

## Custom PQMF

In [ ]:
from thesis import pqmf1

# audio_torch = torch.Tensor(audio_np).reshape((1, 1, -1))
p1 = pqmf1.PQMF(attenuation=attenuation, n_band=bands)

audio_tf1 = tf.convert_to_tensor(audio_np)
audio_tf1 = audio_tf1.reshape((1, -1, 1))

In [ ]:
x = tf.convert_to_tensor(audio_np)
print(x.shape)

import einops
x = einops.rearrange(x, "(b t) -> b t 1", b=2)

play_audio(x[1])

In [ ]:
audio_tf1.

In [ ]:
temp1 = p1.analysis(audio_tf1)

temp1.shape

In [ ]:
# Upsample to hear what it sounds like

from thesis.newt import resample

a2 = resample(temp, temp.shape[1] * 8)

print(a2.shape)

for i in range(8):
    play_audio(a2[:,:sample_rate*8,i])

In [ ]:
rec1 = p1.synthesis(temp)
rec1.shape

In [ ]:
play_audio(rec1)
get_loss(audio_tf1, rec1)

## Comparison

In [ ]:
t0 = np.array(temp0).swapaxes(1, 2)
t1 = np.array(temp1)

In [ ]:
print(((t0 - t1) ** 2).mean())
print(((t0 - t1) ** 2).max())

In [ ]:
((np.array(p1.hk).squeeze().T - np.array(p0.hk)) ** 2).sum()

In [ ]:
p0.hk

In [ ]:
rec2 = p1.synthesis(t0)
print(rec2.shape)
play_audio(rec2)
get_loss(audio_tf1, rec2)

In [ ]:
rec3 = p0.inverse(torch.Tensor(t1.swapaxes(1, 2)))
play_audio(rec3)
get_loss(audio_tf1, rec3.swapaxes(1, 2))

In [ ]:
x = np.zeros(30)
x[::5] = 1
x

## Debugging

In [ ]:
x = t0
x1 = tf.nn.conv1d_transpose(
    x,
    p1.updown_filter * p1.n_band,
    strides=p1.n_band,
    output_shape=(
        tf.shape(x)[0],
        tf.shape(x)[1] * p1.n_band,
        p1.n_band,
    ),
)

x1_torch = torch.Tensor(np.array(x1))

In [ ]:
hk = p0.hk.flip(-1)
y = nn.functional.conv1d(
        x1_torch,
        p0.hk.flip(-1).unsqueeze(0),
        padding=hk.shape[-1] // 2,
    )[..., 1:]